<a href="https://colab.research.google.com/github/tejasuvarna/launch/blob/master/Collab_Cheatsheet_Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important Topics Covered in Cheatsheet**
1. Scaling Numerical Data Values
2. Imputation of Categorical Values
3. Multicollinearity Techniques
4. Support Vector Machine Code
5. Training and Test Data Splie
6. Random Forest Model
   

In [0]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as smd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.impute import SimpleImputer

In [30]:
#Uploading files
from google.colab import files
uploaded = files.upload()

Saving bank-test.csv to bank-test (1).csv
Saving bank-train.csv to bank-train (1).csv


In [0]:
bank_Train = pd.read_csv("bank-train.csv")
bank_Test = pd.read_csv("bank-test.csv")

In [32]:
#SCALING NUMERICAL VALUES
from sklearn.preprocessing import StandardScaler
bank_Train_Numerical = bank_Train.select_dtypes(exclude=['object'])
scaler = StandardScaler()
scaler.fit(bank_Train_Numerical)
scaled = scaler.transform(bank_Train_Numerical)
b_T_N = pd.DataFrame(scaled, index=bank_Train_Numerical.index, columns=bank_Train_Numerical.columns) 
bank_Train.update(b_T_N)
print (bank_Train.head())

         id       age          job  ... euribor3m nr.employed         y
0 -0.678621 -0.001669  blue-collar  ...  0.771354    0.843165 -0.355933
1  1.249788 -0.864095       admin.  ... -1.372028   -0.939748 -0.355933
2  0.840522  1.819007      retired  ... -1.308581   -0.939748 -0.355933
3 -0.227326  0.285806    housemaid  ...  0.771930    0.843165 -0.355933
4 -1.456976 -0.097494       admin.  ...  0.713674    0.330405 -0.355933

[5 rows x 22 columns]


In [0]:
#IMPUTATION OF CATEGORICAL VARIABLES
bank_Train_Categorical = bank_Train.select_dtypes(include=['object'])
bank_Train_Y = bank_Train['y']
bank_Train_Categorical = pd.concat([bank_Train_Categorical, bank_Train_Y], axis=1)

bank_Test_Categorical = bank_Test.select_dtypes(include=['object'])

imp = SimpleImputer(missing_values="unknown", strategy='most_frequent')
bank_Train_Categorical = pd.DataFrame(imp.fit_transform(bank_Train_Categorical))
bank_Test_Categorical = pd.DataFrame(imp.fit_transform(bank_Test_Categorical))
train = pd.concat([bank_Train_Categorical, bank_Train_Numerical], axis=1, sort=True)
bank_Train_Numerical = train.select_dtypes(exclude=['object'])
bank_Train_Categorical = train.select_dtypes(include=['object'])
bank_Train_Categorical
bank_Train_Categorical.columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']
bank_Test_Categorical.columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [34]:
#SOLVING FOR MULTICOLLINEARITY
#This was done after scaling data. It is important to keep in mind that to find the accurate VIF scores for columns in a dataframe
#they must be calculated before scaling the dataset

"Calculating Variance Inflation Factor" 
#drop when VIF is greater than 2.5 or if R^2 is greater than 0.6 approximately
X = add_constant(bank_Train_Numerical) 
multicollTable = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)
multicollTable

    

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


const             571370.999805
id                     5.004084
age                    1.020918
duration               1.230222
campaign               1.042304
pdays                  1.686819
previous               1.807939
emp.var.rate          35.061847
cons.price.idx         7.628386
cons.conf.idx          2.691016
euribor3m             68.213480
nr.employed           31.796793
y                      1.509852
dtype: float64

In [35]:
#GETTING DUMMY DATA
bank_Train_Categorical_Dummy = pd.get_dummies(bank_Train_Categorical)
bank_Train_Categorical_Dummy.dtypes
bank_Test_Categorical_Dummy = pd.get_dummies(bank_Test_Categorical)
bank_Test_Categorical_Dummy.dtypes

job_admin.                       uint8
job_blue-collar                  uint8
job_entrepreneur                 uint8
job_housemaid                    uint8
job_management                   uint8
job_retired                      uint8
job_self-employed                uint8
job_services                     uint8
job_student                      uint8
job_technician                   uint8
job_unemployed                   uint8
marital_divorced                 uint8
marital_married                  uint8
marital_single                   uint8
education_basic.4y               uint8
education_basic.6y               uint8
education_basic.9y               uint8
education_high.school            uint8
education_illiterate             uint8
education_professional.course    uint8
education_university.degree      uint8
default_no                       uint8
housing_no                       uint8
housing_yes                      uint8
loan_no                          uint8
loan_yes                 

In [0]:
#TRAINING AND TEST DATA
frames = [bank_Train_Numerical, bank_Train_Categorical_Dummy]    
allTestData = pd.concat(frames, axis=1)
allTestData.dtypes
allTestData_X = allTestData.drop(columns='y')
allTestData_Y = allTestData['y']
allTestData_Y.columns = ["y", "y2"]
allTestData_Y = allTestData_Y.drop(columns=['y2'])
X_train, X_test, y_train, y_test = train_test_split(allTestData_X, allTestData_Y, test_size=0.25)

bank_Test_Numerical = bank_Test.select_dtypes(exclude=['object'])

frames2 = [bank_Test_Numerical, bank_Test_Categorical_Dummy]    
allTestData2 = pd.concat(frames2, axis=1)
allTestData2 = allTestData2.drop(columns=['emp.var.rate', 'cons.price.idx', 'euribor3m', 'nr.employed'])